# Parsing APIs Example

## Intro

Now we will take a look on a real data. When you parse data from web you will often meet API based web-pages. 

For example [zalando.fr](https://www.zalando.fr/accueil-homme/) is API based web-page. 

In this guided lab you will learn how to obtain the links from webpages and extract the data. Read through this doc, execute the cells in order and make sure you understand the explanations. 

*Note: This guided lab uses Google Chrome. Other browsers like Safari and Firefox have similar tools for developers but they work differently. To save your time in following this lab, it is strongly recommended that you install and use Google Chrome.*

## Obtaining the link

Zalando is discount e-store where you can buy clothes and accesories with discount. When we go to the web-page, we can choose different sections. First the general process will be shown using [Children section](https://www.zalando.fr/accueil-enfant/) as example.

Here we will parse data about promotions only. Therefore, final output will be the DataFrame with all the goods under discount.

[![Image from Gyazo](https://i.gyazo.com/fa4874d8e81c7570273bbfb853d66308.png)](https://gyazo.com/fa4874d8e81c7570273bbfb853d66308)


We go to Promos page. Right click of mouse shows us a list of actions possible, from which we select Inspect.

<img src='https://i.gyazo.com/bccbd11d69c9040dc98758d443e32052.png' width="400">


You will see the menu dropdown on the right side or on the bottom of the window. There you should click on Network:


[![Image from Gyazo](https://i.gyazo.com/f7e0db81cbfee67694183d1a7640bf81.png)](https://gyazo.com/f7e0db81cbfee67694183d1a7640bf81)

Right after the developer part will change showing the files behind the page. In order to obtain only useful files we select the following settings:
1. Preserve Log
2. Select XHR files.

[![Image from Gyazo](https://i.gyazo.com/9a899d4441d9d93e795f79747f1e47d5.png)](https://gyazo.com/9a899d4441d9d93e795f79747f1e47d5)

In order to obtain some files we need to scrool down and go forward to second page. 

[![Image from Gyazo](https://i.gyazo.com/0956eb3d5125075a236c9a439c7749c7.png)](https://gyazo.com/0956eb3d5125075a236c9a439c7749c7)

In the Network panel you can see the following files being uploaded. All the data on the web-page is uploaded from the json file, which is one of the following. It is important to understand which file contains what kind of information. 

<a href="https://gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c"><img src="https://i.gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c.png" alt="Image from Gyazo" width="724.8"/></a>

When you find what kind of information you need for the data to be uploaded you just test it. Here we need the article... file:

<a href="https://gyazo.com/78b35bf492994b3f35c0564a21da202a"><img src="https://i.gyazo.com/78b35bf492994b3f35c0564a21da202a.png" alt="Image from Gyazo" width="727.2"/></a>

When we test the link in Chrome inkognito mode we obtain the proper json file:


<a href="https://gyazo.com/b60453fa98454fa29771c731a5174443"><img src="https://i.gyazo.com/b60453fa98454fa29771c731a5174443.png" alt="Image from Gyazo" width="1530.4"/></a>

In order to change the objects in the json file (kind of pagination), you need to change the offset (the number of the first element on the page). in fact, if you take a look on the link, it is easy to unerstand the structure of the link.

# Reading the data

Now the party rocks! When we know how can we obtain the data, it is not a problem to obtain the whole database with all the data from the web-page.
In this lab you will collect your database of Zalando products. You select which goods you want to track. You can define as many filters to your data as you want. Just make sure that the data represents the filters.




In [49]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize


In [50]:
# Paste the url you obtained for your data
url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale'


#### Collect first 84 object of the of the data (1st page)

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having *family_articles, flags, media* and *sizes* remaining lists (they are exceptions).

In [51]:
# Your code
df = pd.read_json(url)

response = requests.get(url)
results = response.json()
results


flattened_data = json_normalize(results)

flattened_data1 = json_normalize(flattened_data.articles[0])
flattened_data1

HTTPError: HTTP Error 410: Gone

#### Collect all the objects from selected filters. Total number of pages can be found in the same json. Use *sku* column as index.

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having family_articles, flags, media and sizes remaining lists (they are exceptions).

In [ ]:
# Get the total number of pages
total_pages=results['pagination']['page_count']

# Your code
df=pd.DataFrame()
for i in range(total_pages):
    k=84*i
    url=f'https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset={k}&sort=sale'
    response = requests.get(url)
    results = response.json()
    flattened_data = json_normalize(results)
    flattened_data1 = json_normalize(flattened_data.articles[0])
    flattened_data1=flattened_data1.set_index('sku')
    df = df.append(flattened_data1)

df

KeyError: 'pagination'

#### Display the trending brand in DataFrame

In [ ]:
df.brand_name.value_counts().index[0]

AttributeError: 'DataFrame' object has no attribute 'brand_name'

#### Display the brand with maximal total discount (sum of discounts on all goods)

In [ ]:
#Our data is still text. Convert prices into numbers:
df['price.original']=df['price.original'].str.extract('(\d*,\d*)')
df['price.promotional']=df['price.promotional'].str.extract('(\d*,\d*)')

df['price.original'] = [x.replace(',', '.') for x in df['price.original']]
df['price.promotional'] = [x.replace(',', '.') for x in df['price.promotional']]

In [ ]:
df['discount_amount']=df['price.original'].astype(float)-df['price.promotional'].astype(float)
df1=df.copy()

In [ ]:
total_disc=df1.groupby(['brand_name']).sum().discount_amount

In [ ]:
total_disc.sort_values(ascending=False).index[0]

#### Display the brands without discount at all

In [ ]:
total_disc[total_disc==0]

In [52]:
import http.client

conn = http.client.HTTPSConnection("wrist-watches.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "2fd8afb786msh851c73f856e5585p19bd24jsn57cc8a805507",
    'X-RapidAPI-Host': "wrist-watches.p.rapidapi.com"
    }

conn.request("GET", "/watches", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

df = pd.read_json(data.decode("utf-8"))

df.head(100)

[{"id":1,"name":"Reflex Vox Digital Silicone Red Strap Watch","price":"65.32","image":"https://staticimg.titan.co.in/Fastrack/Catalog/38072AP04_1.jpg?pView=pdp","desc":"astrack Reflex Vox is the First smartwatch from the brand Fastrack . It comes in 4 unique colors - Carbon Black, Flaming Red, Champagne Pink & Blazing Blue. Fastrack Reflex Vox comes with a 1.69... Large Display Screen, Built in Alexa, 100+Watch Face & 10+Sports Mode. Reflex Vox has got a large range of features such as menstrual tracker, Stress Monitor, SPO2 tracker, Sleep Tracker, 24*7 Heart Rate Monitor & 5ATM Water resistant. All these features can be enjoyed with a super lasting battery of 10 days. Reflex Vox comes with an exciting option of interchangeable straps to give a new look every day.","quantity":1,"techSpecs":[{"brand":"fastrack"},{"waterResistant":"5ATM"},{"Collection":"Reflex Vox"},{"Warranty":"12 Months on the date of purchase"},{"Movement":"Smart"},{"strapMaterial":"Silicone"},{"lockMechanism":"Buckle

,id,name,price,image,desc,quantity,techSpecs
0,1,Reflex Vox Digital Silicone Red Strap Watch,65.32,https://staticimg.titan.co.in/Fastrack/Catalog...,astrack Reflex Vox is the First smartwatch fro...,1,"[{'brand': 'fastrack'}, {'waterResistant': '5A..."
1,2,Stunners 3.0 Black Dial Metal Strap Watch,33.93,https://staticimg.titan.co.in/Fastrack/Catalog...,not available,1,"[{'brand': 'fastrack'}, {'waterResistant': '5A..."
2,3,After Dark Green Dial Leather Strap Watch,62.70,https://staticimg.titan.co.in/Fastrack/Catalog...,Own the hustle with the new Afterdark collecti...,1,"[{'brand': 'fastrack'}, {'waterResistant': '5A..."
3,4,After Dark Black Dial Leather Strap Watch for ...,53.55,https://staticimg.titan.co.in/Fastrack/Catalog...,Own the hustle with the new Afterdark collecti...,1,"[{'brand': 'fastrack'}, {'waterResistant': '5A..."
4,5,Batman Blue Dial Leather Strap Watch,33.93,https://staticimg.titan.co.in/Fastrack/Catalog...,"For the Batman fan in you, flaunt the perfect ...",1,"[{'brand': 'fastrack'}, {'waterResistant': '5A..."
5,6,Batman Black Dial Stainless Steel Strap Watch,62.70,https://staticimg.titan.co.in/Fastrack/Catalog...,"For the Batman fan in you, flaunt the perfect ...",1,"[{'brand': 'fastrack'}, {'waterResistant': '5A..."
6,7,Batman Black Dial Leather Strap Watch,41.78,https://staticimg.titan.co.in/Fastrack/Catalog...,"For the Batman fan in you, flaunt the perfect ...",1,"[{'brand': 'fastrack'}, {'waterResistant': '5A..."
7,8,Urban Bounce Red Dial Red Strap Watch,54.85,https://staticimg.titan.co.in/Fastrack/Catalog...,no description available,1,"[{'brand': 'fastrack'}, {'waterResistant': '5A..."
8,9,Urban Bounce Red Dial Red Strap Watch,54.85,https://staticimg.titan.co.in/Fastrack/Catalog...,no description available,1,"[{'brand': 'fastrack'}, {'waterResistant': '5A..."
9,10,Wonder Woman Black Dial Leather Strap Watch,49.51,https://staticimg.titan.co.in/Fastrack/Catalog...,"For the Wonder woman fan in you, flaunt the pe...",1,"[{'brand': 'fastrack'}, {'waterResistant': '5A..."


In [53]:
import http.client

conn = http.client.HTTPSConnection("mental-health-info-api.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "2fd8afb786msh851c73f856e5585p19bd24jsn57cc8a805507",
    'X-RapidAPI-Host': "mental-health-info-api.p.rapidapi.com"
    }

conn.request("GET", "/news/thetimes", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

[{"title":"Stagnating wages linked to rise in mental health problemsWages in Scotland have experienced “unprecedented stagnation” over the past 12 years and this has led to stalling...Monday November 14 2022, 12.01am, The TimesLauren Gilmour, Dan Barker","url":"https://www.thetimes.co.uk/article/stagnating-wages-linked-to-rise-in-mental-health-problems-sjzzhbqzh","source":"thetimes"},{"title":"BODY & SOULJuliet Stevenson: I turned off Twitter and my mental health transformedThe actress Juliet Stevenson, 66 tomorrow, quit Twitter three years ago after a series of increasingly impassioned...Saturday October 29 2022, 12.01am, The TimesJulia Llewellyn Smith","url":"https://www.thetimes.co.uk/article/juliet-stevenson-interview-whkcfnv92","source":"thetimes"},{"title":"IN CONVERSATION | MARK BEATTIE‘Right to jury trial is a pretty fundamental part of our democracy’Mark Beattie saw an advert on the side of a bus 16 years ago that was a pitch for the recruitment of lay...Thursday October 27 20

In [54]:
df = pd.read_json(data.decode("utf-8"))

df.head()

,title,url,source
0,Stagnating wages linked to rise in mental heal...,https://www.thetimes.co.uk/article/stagnating-...,thetimes
1,BODY & SOULJuliet Stevenson: I turned off Twit...,https://www.thetimes.co.uk/article/juliet-stev...,thetimes
2,IN CONVERSATION | MARK BEATTIE‘Right to jury t...,https://www.thetimes.co.uk/article/right-to-ju...,thetimes
3,BOOKSRest easy with a dream teamTravelling ins...,https://www.thetimes.co.uk/article/rest-easy-w...,thetimes
4,RED BOX | KELLY HOLMESWe need a new plan for m...,https://www.thetimes.co.uk/article/we-need-a-n...,thetimes


In [59]:
import http.client

conn = http.client.HTTPSConnection("coinranking1.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "2fd8afb786msh851c73f856e5585p19bd24jsn57cc8a805507",
    'X-RapidAPI-Host': "coinranking1.p.rapidapi.com"
    }

conn.request("GET", "/coins?referenceCurrencyUuid=yhjMzLPhuIDl&timePeriod=24h&tiers%5B0%5D=1&orderBy=marketCap&orderDirection=desc&limit=50&offset=0", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"status":"success","data":{"stats":{"total":1479,"totalCoins":22200,"totalMarkets":34936,"totalExchanges":184,"totalMarketCap":"890902191144","total24hVolume":"41645853657"},"coins":[{"uuid":"Qwsogvtv82FCd","symbol":"BTC","name":"Bitcoin","color":"#f7931A","iconUrl":"https://cdn.coinranking.com/bOabBYkcX/bitcoin_btc.svg","marketCap":"325833021714","price":"16953.108837894153","listedAt":1330214400,"tier":1,"change":"-0.92","rank":1,"sparkline":["17132.639817480067","17145.86903708733","17146.392558901003","17117.091160112177","17113.93342205491","17100.078865378524","17109.067795665014","17103.125654790616","17088.113276149066","17106.00172508932","17139.31133327529","17093.30285052494","17073.31813817331","16992.855657142885","16983.407441631334","16983.222802675773","16953.134187909865","16943.373527731288","16937.975710095026","16920.33373863517","16936.98572546867","16947.907762284987","16968.142789929694","16982.22235576328","16958.234039502968"],"lowVolume":false,"coinrankingUrl

In [60]:
import requests

url = "https://coinranking1.p.rapidapi.com/coins"

querystring = {"referenceCurrencyUuid":"yhjMzLPhuIDl","timePeriod":"24h","tiers[0]":"1","orderBy":"marketCap","orderDirection":"desc","limit":"50","offset":"0"}

headers = {
	"X-RapidAPI-Key": "2fd8afb786msh851c73f856e5585p19bd24jsn57cc8a805507",
	"X-RapidAPI-Host": "coinranking1.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

# display the data in a dictionary format 
response.json()


{"status":"success","data":{"stats":{"total":1483,"totalCoins":22200,"totalMarkets":34937,"totalExchanges":184,"totalMarketCap":"890849144897","total24hVolume":"41645293813"},"coins":[{"uuid":"Qwsogvtv82FCd","symbol":"BTC","name":"Bitcoin","color":"#f7931A","iconUrl":"https://cdn.coinranking.com/bOabBYkcX/bitcoin_btc.svg","marketCap":"325813753132","price":"16952.106292613524","listedAt":1330214400,"tier":1,"change":"-0.96","rank":1,"sparkline":["17134.17374588722","17146.87978152998","17144.316207582313","17115.381922448803","17116.20878147723","17098.025896350668","17109.291170395056","17102.226632860245","17087.273497110466","17109.42369869617","17140.252581722678","17089.26427945373","17073.31813817331","16992.855657142885","16983.407441631334","16983.222802675773","16953.134187909865","16943.373527731288","16937.975710095026","16920.33373863517","16936.98572546867","16947.907762284987","16968.142789929694","16982.22235576328","16958.234039502968"],"lowVolume":false,"coinrankingUrl

{'status': 'success',
 'data': {'stats': {'total': 1483,
   'totalCoins': 22200,
   'totalMarkets': 34937,
   'totalExchanges': 184,
   'totalMarketCap': '890849144897',
   'total24hVolume': '41645293813'},
  'coins': [{'uuid': 'Qwsogvtv82FCd',
    'symbol': 'BTC',
    'name': 'Bitcoin',
    'color': '#f7931A',
    'iconUrl': 'https://cdn.coinranking.com/bOabBYkcX/bitcoin_btc.svg',
    'marketCap': '325813753132',
    'price': '16952.106292613524',
    'listedAt': 1330214400,
    'tier': 1,
    'change': '-0.96',
    'rank': 1,
    'sparkline': ['17134.17374588722',
     '17146.87978152998',
     '17144.316207582313',
     '17115.381922448803',
     '17116.20878147723',
     '17098.025896350668',
     '17109.291170395056',
     '17102.226632860245',
     '17087.273497110466',
     '17109.42369869617',
     '17140.252581722678',
     '17089.26427945373',
     '17073.31813817331',
     '16992.855657142885',
     '16983.407441631334',
     '16983.222802675773',
     '16953.134187909865',


In [61]:
import http.client

conn = http.client.HTTPSConnection("walmart.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "2fd8afb786msh851c73f856e5585p19bd24jsn57cc8a805507",
    'X-RapidAPI-Host': "walmart.p.rapidapi.com"
    }

conn.request("GET", "/v2/auto-complete?term=blanket", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

import json
data = json.loads(data.decode("utf-8"))
data




{"queries":[{"type":"QUERY","displayName":"blanket","url":"query=blanket&typeahead=blanket"},{"type":"QUERY","displayName":"blanket","url":"query=blanket&typeahead=blanket","metaData":{"deptId":"4044","deptName":"Home"}},{"type":"QUERY","displayName":"blankets and throws","url":"query=blankets%20and%20throws&typeahead=blanket"},{"type":"QUERY","displayName":"throw blanket","url":"query=throw%20blanket&typeahead=blanket"},{"type":"QUERY","displayName":"electric blanket","url":"query=electric%20blanket&typeahead=blanket"},{"type":"QUERY","displayName":"blankets for beds","url":"query=blankets%20for%20beds&typeahead=blanket"},{"type":"QUERY","displayName":"heated blanket","url":"query=heated%20blanket&typeahead=blanket"},{"type":"QUERY","displayName":"weighted blanket","url":"query=weighted%20blanket&typeahead=blanket"},{"type":"QUERY","displayName":"fleece blanket","url":"query=fleece%20blanket&typeahead=blanket"}],"fns":{"navigationModules":[]}}


{'queries': [{'type': 'QUERY',
   'displayName': 'blanket',
   'url': 'query=blanket&typeahead=blanket'},
  {'type': 'QUERY',
   'displayName': 'blanket',
   'url': 'query=blanket&typeahead=blanket',
   'metaData': {'deptId': '4044', 'deptName': 'Home'}},
  {'type': 'QUERY',
   'displayName': 'blankets and throws',
   'url': 'query=blankets%20and%20throws&typeahead=blanket'},
  {'type': 'QUERY',
   'displayName': 'throw blanket',
   'url': 'query=throw%20blanket&typeahead=blanket'},
  {'type': 'QUERY',
   'displayName': 'electric blanket',
   'url': 'query=electric%20blanket&typeahead=blanket'},
  {'type': 'QUERY',
   'displayName': 'blankets for beds',
   'url': 'query=blankets%20for%20beds&typeahead=blanket'},
  {'type': 'QUERY',
   'displayName': 'heated blanket',
   'url': 'query=heated%20blanket&typeahead=blanket'},
  {'type': 'QUERY',
   'displayName': 'weighted blanket',
   'url': 'query=weighted%20blanket&typeahead=blanket'},
  {'type': 'QUERY',
   'displayName': 'fleece blanke

In [62]:
import http.client

conn = http.client.HTTPSConnection("walmart.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "2fd8afb786msh851c73f856e5585p19bd24jsn57cc8a805507",
    'X-RapidAPI-Host': "walmart.p.rapidapi.com"
    }

conn.request("GET", "/stores/list?postalCode=52774", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

import json
data = json.loads(data.decode("utf-8"))
data



{"data":{"storesBySearchTerm":{"stores":[{"distance":10.19,"id":1509,"displayName":"Maquoketa Supercenter","storeType":{"id":"1","displayName":"Supercenter"},"address":{"postalCode":"52060","address":"103 E Carlisle","city":"Maquoketa","state":"IA","country":"US"},"operationalHours":{"open24Hours":null,"todayHrs":{"startHr":"06:00","endHr":"23:00","openFullDay":null},"tomorrowHrs":{"startHr":"06:00","endHr":"23:00","openFullDay":null}},"geoPoint":{"latitude":42.048737,"longitude":-90.662622},"timeZone":"America/Chicago"},{"distance":19.25,"id":2889,"displayName":"Clinton Supercenter","storeType":{"id":"1","displayName":"Supercenter"},"address":{"postalCode":"52732","address":"2715 S 25th St","city":"Clinton","state":"IA","country":"US"},"operationalHours":{"open24Hours":null,"todayHrs":{"startHr":"06:00","endHr":"23:00","openFullDay":null},"tomorrowHrs":{"startHr":"06:00","endHr":"23:00","openFullDay":null}},"geoPoint":{"latitude":41.8186521,"longitude":-90.2426849},"timeZone":"America

{'data': {'storesBySearchTerm': {'stores': [{'distance': 10.19,
     'id': 1509,
     'displayName': 'Maquoketa Supercenter',
     'storeType': {'id': '1', 'displayName': 'Supercenter'},
     'address': {'postalCode': '52060',
      'address': '103 E Carlisle',
      'city': 'Maquoketa',
      'state': 'IA',
      'country': 'US'},
     'operationalHours': {'open24Hours': None,
      'todayHrs': {'startHr': '06:00', 'endHr': '23:00', 'openFullDay': None},
      'tomorrowHrs': {'startHr': '06:00',
       'endHr': '23:00',
       'openFullDay': None}},
     'geoPoint': {'latitude': 42.048737, 'longitude': -90.662622},
     'timeZone': 'America/Chicago'},
    {'distance': 19.25,
     'id': 2889,
     'displayName': 'Clinton Supercenter',
     'storeType': {'id': '1', 'displayName': 'Supercenter'},
     'address': {'postalCode': '52732',
      'address': '2715 S 25th St',
      'city': 'Clinton',
      'state': 'IA',
      'country': 'US'},
     'operationalHours': {'open24Hours': None,
  

In [63]:
import http.client

conn = http.client.HTTPSConnection("walmart.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "2fd8afb786msh851c73f856e5585p19bd24jsn57cc8a805507",
    'X-RapidAPI-Host': "walmart.p.rapidapi.com"
    }

conn.request("GET", "/v2/auto-complete?term=hot%20wheels", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

import json
data = json.loads(data.decode("utf-8"))
data

{"queries":[{"type":"QUERY","displayName":"hot wheels","url":"query=hot%20wheels&typeahead=hot%20wheels"},{"type":"QUERY","displayName":"hot wheels","url":"query=hot%20wheels&typeahead=hot%20wheels","metaData":{"deptId":"4171","deptName":"Toys"}},{"type":"QUERY","displayName":"hot wheels track","url":"query=hot%20wheels%20track&typeahead=hot%20wheels"},{"type":"QUERY","displayName":"hot wheels cars","url":"query=hot%20wheels%20cars&typeahead=hot%20wheels"},{"type":"QUERY","displayName":"hot wheels monster trucks","url":"query=hot%20wheels%20monster%20trucks&typeahead=hot%20wheels"},{"type":"QUERY","displayName":"hot wheels ultimate garage","url":"query=hot%20wheels%20ultimate%20garage&typeahead=hot%20wheels"},{"type":"QUERY","displayName":"hot wheels garage","url":"query=hot%20wheels%20garage&typeahead=hot%20wheels"},{"type":"QUERY","displayName":"hot wheels mario kart rainbow road race track","url":"query=hot%20wheels%20mario%20kart%20rainbow%20road%20race%20track&typeahead=hot%20whee

{'queries': [{'type': 'QUERY',
   'displayName': 'hot wheels',
   'url': 'query=hot%20wheels&typeahead=hot%20wheels'},
  {'type': 'QUERY',
   'displayName': 'hot wheels',
   'url': 'query=hot%20wheels&typeahead=hot%20wheels',
   'metaData': {'deptId': '4171', 'deptName': 'Toys'}},
  {'type': 'QUERY',
   'displayName': 'hot wheels track',
   'url': 'query=hot%20wheels%20track&typeahead=hot%20wheels'},
  {'type': 'QUERY',
   'displayName': 'hot wheels cars',
   'url': 'query=hot%20wheels%20cars&typeahead=hot%20wheels'},
  {'type': 'QUERY',
   'displayName': 'hot wheels monster trucks',
   'url': 'query=hot%20wheels%20monster%20trucks&typeahead=hot%20wheels'},
  {'type': 'QUERY',
   'displayName': 'hot wheels ultimate garage',
   'url': 'query=hot%20wheels%20ultimate%20garage&typeahead=hot%20wheels'},
  {'type': 'QUERY',
   'displayName': 'hot wheels garage',
   'url': 'query=hot%20wheels%20garage&typeahead=hot%20wheels'},
  {'type': 'QUERY',
   'displayName': 'hot wheels mario kart rainb